In [ ]:
import os

from urllib.request import urlopen
import json
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import timedelta
import holidays

from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import make_pipeline
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import r2_score
from sklearn.inspection import permutation_importance


from pandas.tseries.holiday import AbstractHolidayCalendar, Holiday, nearest_workday, \
    USMartinLutherKingJr, USPresidentsDay, GoodFriday, USMemorialDay, \
    USLaborDay, USThanksgivingDay

import warnings; warnings.simplefilter('ignore')

from scipy import stats

In [ ]:
def update_progress(progress):
    bar_length = 20
    if isinstance(progress, int):
        progress = float(progress)
    if not isinstance(progress, float):
        progress = 0
    if progress < 0:
        progress = 0
    if progress >= 1:
        progress = 1

    block = int(round(bar_length * progress))

    text = "Progress: [{0}] {1:.3f}%".format( "#" * block + "-" * (bar_length - block), progress * 100)
    print(text, end="\r")

# Util Functions

In [ ]:
class USTradingCalendar(AbstractHolidayCalendar):
    rules = [
        Holiday('NewYearsDay', month=1, day=1, observance=nearest_workday),
        USMartinLutherKingJr,
        USPresidentsDay,
        GoodFriday,
        USMemorialDay,
        Holiday('USIndependenceDay', month=7, day=4, observance=nearest_workday),
        USLaborDay,
        USThanksgivingDay,
        Holiday('Christmas', month=12, day=25, observance=nearest_workday)
    ]


def get_trading_close_holidays(strt, end):
    inst = USTradingCalendar()

    return inst.holidays(datetime(strt, 12, 31), datetime(end, 12, 31))


def get_days_between(date1, date2):
    date_format = "%Y-%m-%d"
    a = datetime.strptime(date1, date_format)
    b = datetime.strptime(date2, date_format)
    delta = b - a
    return delta.days


def is_holiday(d):
    return (d in get_trading_close_holidays(d.year-1, d.year))


def next_weekday(date1):
    
    date_format = "%Y-%m-%d"
    d = datetime.strptime(date1, date_format)
    
    while is_holiday(d) or (d.weekday() >=5):
            d = d - timedelta(1)
    
    return d.strftime(date_format)

# API Helper Functions

In [ ]:
def get_built_url(query):
    url = "https://financialmodelingprep.com/api/v3/"
    api_key = "apikey=XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"
    
    return url + query + api_key


def get_jsonparsed_data(url):
    """
    Receive the content of ``url``, parse it as JSON and return the object.

    Parameters
    ----------
    url : str

    Returns
    -------
    dict
    """
    response = urlopen(url)
    data = response.read().decode("utf-8")
    return json.loads(data)


def get_df_from_url(url):
    return pd.DataFrame.from_dict(get_jsonparsed_data(url))

# Reduced FMP API Library

In [ ]:
def get_all_symbols_df():
    """
    Fetches all symbols on FMP, parses them in pandas and return a DataFrame object.

    Parameters
    ----------
    None

    Returns
    -------
    pd.DataFrame
    """
    query = 'available-traded/list?'
    url = get_built_url(query)
    data = get_df_from_url(url)['symbol']
    data.to_pickle("./all_symbols.pkl")
    
    return data


def get_ratios_df(symbol='AAPL'):
    """
    Fetches ratios for ``symbol``, parse it in pandas and return a DataFrame object.

    Parameters
    ----------
    symbol : str

    Returns
    -------
    pd.DataFrame
    """
    query = "ratios/"+symbol+"?period=quarter&"
    url = get_built_url(query)
    ret_df = get_df_from_url(url)
#     ret_df.set_index('date', inplace=True)
    return ret_df


def get_key_metrics_df(symbol='AAPL'):
    """
    Fetches key-metrics for ``symbol``, parse it in pandas and return a DataFrame object.

    Parameters
    ----------
    symbol : str

    Returns
    -------
    pd.DataFrame
    """
    query = "key-metrics/"+symbol+"?period=quarter&"
    url = get_built_url(query)
    ret_df = get_df_from_url(url)
#     ret_df.set_index('date', inplace=True)
    return ret_df


def get_company_profile_df(symbol='AAPL'):
    """
    Fetches profile for ``symbol``, parse it in pandas and return a DataFrame object.

    Parameters
    ----------
    symbol : str

    Returns
    -------
    pd.DataFrame
    """
    query = "profile/"+symbol+"?"
    url = get_built_url(query)
    return get_df_from_url(url)


def fetch_save_exchange_profiles(symbol='NYSE'):
    """
    Fetches all profiles for exchange =>``symbol``, parse it in pandas and return a DataFrame object.

    Parameters
    ----------
    symbol : str

    Returns
    -------
    pd.DataFrame
    """
    query = "stock-screener?exchange="+symbol+"&isActivelyTrading=true&isEtf=false&"
    url = get_built_url(query)
    data = get_df_from_url(url)
    
    if not os.path.exists('exchanges'):
        os.makedirs('exchanges')
    data.to_csv('exchanges/'+symbol+'.csv')
    
    return data

        
def get_latest_earnings_update_symbols():
    """
    Fetches ...

    Parameters
    ----------
    symbol : str

    Returns
    -------
    pd.DataFrame
    """
    
    start = (datetime.today() - timedelta(45)).strftime('%Y-%m-%d')
    end = (datetime.today() - timedelta(0)).strftime('%Y-%m-%d')

    query = "earning_calendar?from="+start+"&to="+end+"&"
    url = get_built_url(query)
    return get_df_from_url(url)


def get_full_historic_price(symbol='AAPL'):
    """
    Fetches all historic prices for ``symbol``, parse it in pandas and return a DataFrame object.

    Parameters
    ----------
    symbol : str

    Returns
    -------
    pd.DataFrame
    """
    start = "1989-06-20"
    end = datetime.today().strftime('%Y-%m-%d')
    
    query = "historical-discounted-cash-flow-statement/"+symbol+"?period=quarter&"
    url = get_built_url(query)
    ret_df = get_df_from_url(url)
    ret_df['date'] = pd.to_datetime(ret_df['date'])
    ret_df = ret_df.set_index('date').resample('Q').nearest()
    ret_df['open'] = ret_df['price']
    ret_df = ret_df.iloc[::-1]
    ret_df = ret_df.reset_index()
    
    return ret_df

def get_latest_price(symbol='AAPL'):
    query = "quote-short/"+symbol+"?"
    url = get_built_url(query)
    ret_df = get_df_from_url(url)
    
    return ret_df['price'].iloc[0]


def get_latest_bookValue(symbol='AAPL'):
    query = "key-metrics/"+symbol+"?period=quarter&"
    url = get_built_url(query)
    ret_df = get_df_from_url(url)
    
    return ret_df['bookValuePerShare'].iloc[0]


def get_latest_tangibleBookValue(symbol='AAPL'):
    query = "key-metrics/"+symbol+"?period=quarter&"
    url = get_built_url(query)
    ret_df = get_df_from_url(url)
    
    return ret_df['tangibleBookValuePerShare'].iloc[0]


def get_latest_dividendYield(symbol='AAPL'):
    query = "key-metrics/"+symbol+"?period=quarter&"
    url = get_built_url(query)
    ret_df = get_df_from_url(url)
    
    return ret_df['dividendYield'].iloc[0]



# Data Wrangling Functions

In [ ]:
ratios = ['date', 'operatingProfitMargin',
       'netProfitMargin',
       'assetTurnover', 'freeCashFlowOperatingCashFlowRatio',
       'capitalExpenditureCoverageRatio']

key_metrics = ['revenuePerShare', 'netIncomePerShare',
       'operatingCashFlowPerShare', 'freeCashFlowPerShare',
       'tangibleBookValuePerShare',
       'shareholdersEquityPerShare', 'interestDebtPerShare',
       'enterpriseValue', 'evToSales',
       'evToFreeCashFlow',
       'debtToEquity', 'netDebtToEBITDA', 'currentRatio',
       'incomeQuality',
       'researchAndDdevelopementToRevenue',
       'capexToOperatingCashFlow', 'capexToRevenue',
       'roic', 'workingCapital',
       'netCurrentAssetValue',
       'averageReceivables', 'averagePayables', 'averageInventory',
       'receivablesTurnover', 'payablesTurnover',
       'inventoryTurnover']

price = ['date', 'open', 'dcf']

In [ ]:
def get_single_dataset(symbol='AAPL'):
    my_df = pd.concat([get_key_metrics_df(symbol)[key_metrics], get_ratios_df(symbol)[ratios]], axis=1)

    my_df.interpolate(method='cubic', inplace=True)
    
    prices = get_full_historic_price(symbol)[price]
    
    my_df['open'] = prices['open']
    my_df['dcf'] = prices['dcf']
    
    my_df = my_df.iloc[::-1]
    
    my_df = my_df.set_index('date')
    
    return my_df

# Top 100 Companies List by Market Cap.

In [ ]:
exchange_codes = ['SIX', 'LSE', 'EURONEXT', 'NYSE', 'NASDAQ']

for ex in exchange_codes:
    fetch_save_exchange_profiles(ex)

In [ ]:
exchanges_lst = ['exchanges/'+f for f in os.listdir('exchanges/') if os.path.isfile(os.path.join('exchanges/', f))]

In [ ]:
for choice in exchange_codes:

    top_x_companies = 100

    index = [idx for idx, s in enumerate(exchanges_lst) if choice in s][0]
    all_stks = pd.read_csv(exchanges_lst[index], index_col=0).nlargest(top_x_companies, 'marketCap')['symbol'].to_list()

    results = []

    for sym in all_stks:
        update_progress(all_stks.index(sym)/len(all_stks))

        try:
            dataset = get_single_dataset(sym).fillna(0)

            ltp = get_latest_price(sym)
            my_X = dataset.drop(['open'], 1).iloc[-1]

            if my_X.name >= '2021-09-30':

                print(sym)
                print()
                dataset['LABELS'] = dataset['open'].shift(-1)
                dataset = dataset.dropna()

                if len(dataset) > 8:

                    X = dataset.drop(['open', 'LABELS'], 1)
                    y = dataset['LABELS']

                    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

                    regr = RandomForestRegressor(n_estimators=1000, max_depth=1000, random_state=0)
                    cv_results = cross_validate(regr, X_train, y_train, verbose=0, return_estimator=True, cv=10)

                    best_estimators = []
                    test_scores = []

                    for i in range(len(cv_results['estimator'])):
                        y_pred = cv_results['estimator'][i].predict(X_test)
                        conf_mat = r2_score(y_test, y_pred)
                        my_score = (conf_mat - 0.8) / abs(cv_results['test_score'][i] - conf_mat)
                        best_estimators.append(my_score)
                        test_scores.append(conf_mat)

                    best_estimator = best_estimators.index(max(best_estimators))

                    print(sym,
                          my_X.name,
                          round(test_scores[best_estimator]*100, 2),
                          round(cv_results['test_score'][best_estimator]*100, 2),
                          ltp,
                          my_X.dcf,
                          cv_results['estimator'][best_estimator].predict([my_X.fillna(0)])[0]
                          )

                    results.append([sym,
                          my_X.name,
                          round(test_scores[best_estimator]*100, 2),
                          round(cv_results['test_score'][best_estimator]*100, 2),
                          ltp,
                          my_X.dcf,
                          cv_results['estimator'][best_estimator].predict([my_X.fillna(0)])[0]       
                         ])

        except Exception as e:
            with open('errors.txt', 'w') as f:
                print(e, file=f)

    df = pd.DataFrame.from_records(results, columns=['sym', 'date', 'train', 'test', 'ltp', 'dcf', 'pred'])
    df['train'] = df['train'].where(df['train'] > 75, np.nan)
    df['test'] = df['test'].where(df['test'] > 75, np.nan)
    df = df.drop('date', axis=1)
    df = df.dropna()
    
    df['DCF Growth (%)'] = ((df['dcf'] - df['ltp'])/df['ltp'])*100
    df['AI Growth (%)'] = ((df['pred'] - df['ltp'])/df['ltp'])*100
    
    df['H.Mean Growth (%)'] = stats.hmean(df[['DCF Growth (%)', 'AI Growth (%)']].clip(lower=0), axis=1)

    df.to_csv('FMP_AI_results_'+choice+'.csv')